# BRIDGE: M9.3 HyDE → M9.4 Advanced Reranking

## Purpose

**What shifts:** You've mastered vocabulary-mismatch retrieval with HyDE (M9.3), achieving 58% precision improvement by generating hypothetical documents. Now you face a new problem—your hybrid retrieval returns the **right documents but in the wrong order**. M9.4 introduces advanced reranking to optimize ranking with recency, diversity, and personalization signals without violating latency SLAs.

**Why it matters:** 20-30% of production queries with temporal or diverse content suffer from poor ranking. A 2023 CPRA article buried at rank #48 below outdated 2019 GDPR articles costs teams 33 hours/day in manual re-sorting (₹495K/month lost productivity). Advanced reranking solves ordering problems that retrieval quality alone cannot fix.

## Concepts Covered

**Delta from M9.3:**
- Validating hypothesis generator quality (80%+ domain-appropriate)
- Measuring hybrid retriever precision on vocabulary-mismatch queries (P@10 ≥75%)
- Testing query classifier routing accuracy (85%+ factoid detection)
- Verifying semantic cache performance (30%+ hit rate, ≤400ms P95 latency)

**Preview of M9.4:**
- Ensemble cross-encoder systems with voting
- MMR (Maximal Marginal Relevance) for diversity
- Temporal boosting with exponential decay
- Personalized ranking via CTR-based signals

## After Completing This Bridge

You will be able to:
- ✓ Verify your M9.3 HyDE implementation meets minimum quality thresholds for advanced reranking
- ✓ Identify temporal and diversity issues in your top-K retrieval results
- ✓ Assess whether your system needs advanced reranking (or if a single cross-encoder suffices)
- ✓ Understand the four readiness criteria that prevent reranking failures in M9.4

## Context in Track

**Bridge:** Level 3, Module 9.3 → Module 9.4  
**Previous:** M9.3 Hypothetical Document Embeddings (HyDE)  
**Next:** M9.4 Advanced Reranking Strategies  
**Module:** Module 9 - Advanced Retrieval Techniques  
**Duration:** 8-10 minutes

---

## Run Locally

**Windows:**
```powershell
powershell -c "$env:PYTHONPATH='$PWD'; jupyter notebook"
```

**Linux/macOS:**
```bash
PYTHONPATH=$PWD jupyter notebook
```

**Note:** This notebook runs offline-friendly. External service checks (LLM APIs, datasets, cache logs) gracefully skip if unavailable.

---

## Section 1: RECAP - What M9.3 HyDE Shipped

### Achievements from M9.3 Hypothetical Document Embeddings:

✓ **LLM-powered hypothesis generator**  
Built system that transforms user queries into document-style answers, achieving 80%+ domain-appropriate hypotheses with GPT-4o-mini and domain context prompts

✓ **Hybrid retriever with RRF fusion**  
Implemented Reciprocal Rank Fusion combining HyDE and traditional retrieval, improving precision 15-40% on vocabulary-mismatch queries while maintaining performance on well-phrased queries

✓ **Query classifier with adaptive routing**  
Created factoid detection and vocabulary overlap checker (85%+ accuracy), routing only 30-40% of queries to HyDE to avoid latency/cost overhead

✓ **Semantic cache achieving 30-40% hit rate**  
Reduced effective latency from 500ms to 325ms by caching hypotheses for similar queries using embedding similarity threshold

### Key Outcome:
Precision improved 58% for queries where users and documents speak different languages (vocabulary mismatch).

---

## Section 2: Readiness Check #1 - Hypothesis Generator

**Requirement:** Hypothesis generator achieving 80%+ domain-appropriate hypotheses

**Test:** Evaluate 20 queries to verify hypotheses match document style

**Impact:** Advanced reranking builds on retrieval quality—poor hypotheses = poor retrieved docs = nothing to rerank

### Check 1: LLM API Availability

Verify LLM API keys are configured for hypothesis generation. If unavailable, this check skips gracefully (offline-friendly).

In [ ]:
# Readiness Check #1: Hypothesis Generator Quality
import os

# Skip guard: Check if LLM API key is available
api_key = os.getenv("OPENAI_API_KEY") or os.getenv("ANTHROPIC_API_KEY")

if not api_key:
    print("⚠️ Skipping (no LLM API keys)")
else:
    print("✓ LLM API configured")
    # Expected: Test 20 queries, score hypothesis quality
    # Expected: Calculate: (domain_appropriate_count / 20) >= 0.80
    print("# Expected: 16+/20 hypotheses match document style (80%+)")

---

## Section 3: Readiness Check #2 - Hybrid Retriever Precision

**Requirement:** Hybrid retriever precision ≥75% on vocabulary-mismatch queries

**Test:** Run offline eval on 50 mismatch queries, measure P@10 precision

**Impact:** Reranking can only fix ordering, not fix fundamentally poor retrieval

### Check 2: Evaluation Dataset Availability

Look for evaluation dataset with vocabulary-mismatch queries and ground truth labels. Skips if dataset is not present (offline-friendly).

In [ ]:
# Readiness Check #2: Hybrid Retriever Precision
import os

# Skip guard: Check for evaluation dataset
eval_dataset_path = "eval_mismatch_queries.json"

if not os.path.exists(eval_dataset_path):
    print("⚠️ Skipping (no eval dataset at eval_mismatch_queries.json)")
else:
    print(f"✓ Eval dataset found: {eval_dataset_path}")
    # Expected: Load 50 vocab-mismatch queries with ground truth
    # Expected: Run hybrid retrieval (HyDE + traditional + RRF)
    # Expected: Calculate P@10 = relevant_in_top10 / total_queries >= 0.75
    print("# Expected: P@10 >= 0.75 (37.5+/50 queries)")

---

## Section 4: Readiness Check #3 - Query Classifier Routing

**Requirement:** Query classifier routing correctly (85%+ factoid detection accuracy)

**Test:** Test 40 queries (20 factoid, 20 conceptual), verify routing decisions

**Impact:** Temporal reranking hurts factoid queries (e.g., "When did X happen?")—need correct routing

### Check 3: Query Classifier Accuracy

Test query classifier with sample factoid and conceptual queries to verify routing accuracy meets threshold.

In [ ]:
# Readiness Check #3: Query Classifier Routing Accuracy

# Sample test set (offline-friendly: no external calls)
test_queries = {
    "factoid": ["When did GDPR pass?", "Who invented Python?"],
    "conceptual": ["What are data privacy regulations?", "How does machine learning work?"]
}

print("✓ Query classifier test set loaded")
# Expected: Test 40 queries (20 factoid, 20 conceptual)
# Expected: Classify each query, compare with ground truth
# Expected: Accuracy = correct_classifications / 40 >= 0.85
print("# Expected: 34+/40 queries correctly classified (85%+)")

---

## Section 5: Readiness Check #4 - Semantic Cache Performance

**Requirement:** Semantic cache achieving 30%+ hit rate (latency ≤400ms P95)

**Test:** Monitor cache metrics in logs, verify P95 latency under 400ms

**Impact:** Advanced reranking adds +100-200ms—if base latency already high, total becomes unacceptable

### Check 4: Cache Metrics Availability

Parse cache metrics from logs to calculate hit rate and P95 latency. Skips gracefully if logs are unavailable (offline-friendly).

In [ ]:
# Readiness Check #4: Semantic Cache Hit Rate and Latency
import os

# Skip guard: Check for cache metrics log
cache_log_path = "cache_metrics.log"

if not os.path.exists(cache_log_path):
    print("⚠️ Skipping (no cache metrics at cache_metrics.log)")
else:
    print(f"✓ Cache log found: {cache_log_path}")
    # Expected: Parse cache hits/misses
    # Expected: Calculate hit_rate = hits / (hits + misses) >= 0.30
    # Expected: Parse latency samples, calculate P95 <= 400ms
    print("# Expected: Hit rate >= 30% AND P95 latency <= 400ms")

---

## Section 6: CALL-FORWARD - What M9.4 Advanced Reranking Will Introduce

### The Problem Advanced Reranking Solves:

Your hybrid retrieval (decomposition + multi-hop + HyDE) returns the **right documents**—but in the **wrong order**.

**Example Issues:**
- **Recency:** 2019 GDPR article ranked #1, but 2023 CPRA article (more relevant) buried at #48
- **Diversity:** Top-5 results all from same source (redundant, not diverse perspectives)
- **Performance:** Ensemble rerankers add 300-600ms latency, violating SLAs

### M9.4 Advanced Reranking Strategies Will Cover:

**1. Ensemble Cross-Encoder Systems with Voting**
- Combine 2-3 cross-encoder models with different strengths
- Use majority voting or weighted averaging
- Improve accuracy 8-12% while optimizing latency <200ms P95

**2. MMR (Maximal Marginal Relevance) for Diversity**
- Balance relevance vs diversity using MMR algorithm
- Parameter λ controls trade-off (λ=1: pure relevance, λ=0: pure diversity)
- Ensure top-5 results have 3+ unique sources

**3. Temporal Boosting & Personalized Ranking**
- Apply recency scoring with exponential decay
- Learn user preferences from click data (CTR-based personalization)
- Combine 4 signals: relevance + recency + diversity + personalization

### Key Question for M9.4:

**"Your retrieval returns the right documents. But how do you rank them optimally considering recency, diversity, and user preferences—without adding 500ms latency?"**

### Reality Check:

80-90% of use cases **don't need** advanced reranking. A single cross-encoder is sufficient if your content is:
- Evergreen (doesn't change over time)
- Naturally diverse (no redundancy in top results)
- Used by anonymous users (no personalization needed)

Advanced reranking is specifically for:
- News/regulatory content (recency critical)
- Research/analysis (diversity valuable)
- Personalized systems (user preferences matter)

---

**Bridge Complete! Ready for M9.4 Advanced Reranking Strategies.**